In [8]:
!pip3 install moviepy
# !brew install ffmpeg
!pip3 install decorator==4.0.2
!pip3 install chardet requests==2.25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 2.0 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 2.3 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.1
    Uninstalling urllib3-2.2.1:
      Successfully uninstalled urllib3-2.2.1
  Attempting uninstall: idna
    Found existing installation: idna 3.7
    Uninstalling idna-3.7:
      Successfully uninstalled idna-3.7
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uni

In [34]:
import moviepy.config as cfg
cfg.change_settings({"FFMPEG_BINARY": "/opt/homebrew/bin/ffmpeg"})

In [9]:
import asyncio
import os
from moviepy.editor import VideoFileClip

class VideoProcessingWorkflow:
    def __init__(self, video_path, output_dir):
        self.video_path = video_path
        self.output_dir = output_dir
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        if not os.path.exists(os.path.join(output_dir, "videos")):
            os.makedirs(os.path.join(output_dir, "videos"))
        if not os.path.exists(os.path.join(output_dir, "audios")):
            os.makedirs(os.path.join(output_dir, "audios"))

    async def process_subclip(self, start_time, end_time, subclip_path, audio_path):
        """Process a video subclip asynchronously."""
        try:
            with VideoFileClip(self.video_path).subclip(start_time, end_time) as clip:
                clip.write_videofile(subclip_path, codec='mpeg4')  # Synchronous blocking operation for video
                clip.audio.write_audiofile(audio_path)  # Synchronous blocking operation for audio
                print(f"Processed video and audio from {start_time} to {end_time} seconds.")
        except Exception as e:
            print(f"Error processing media from {start_time} to {end_time} seconds: {str(e)}")

    async def split_video_and_audio(self):
        """Asynchronously splits video into 20-second clips and extracts audio."""
        try:
            with VideoFileClip(self.video_path) as clip:
                duration = int(clip.duration)
                tasks = []
                for start_time in range(0, duration, 20):
                    end_time = min(start_time + 20, duration)
                    subclip_path = os.path.join(self.output_dir, "videos", f"video_{start_time}_{end_time}.mp4")
                    audio_path = os.path.join(self.output_dir, "audios", f"audio_{start_time}_{end_time}.mp3")
                    # Schedule the processing of each subclip
                    task = asyncio.create_task(self.process_subclip(start_time, end_time, subclip_path, audio_path))
                    tasks.append(task)
                # Wait for all scheduled tasks to complete
                await asyncio.gather(*tasks)
        except Exception as e:
            print(f"Error opening video file: {str(e)}")

# This function wraps the workflow initialization and execution
async def main():
    source = "sources/bahubali/"
    video_path = source + "video.mp4"
    output_dir = source + "chunks"
    workflow = VideoProcessingWorkflow(video_path, output_dir)
    await workflow.split_video_and_audio()

# Running the main function in Jupyter's existing event loop
await main()  # Use this line directly in a Jupyter cell

RuntimeError: asyncio.run() cannot be called from a running event loop